In [704]:
# !pip install docplex
# !pip install qiskit

In [705]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse.csgraph import laplacian
import copy

In [706]:
###주소 변경
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MLdata/WDBCdata_V/wdbc.csv')

# df = pd.read_csv('./wdbc.csv')


In [707]:
df.shape

(569, 31)

In [708]:
#df matrix로 변환
df = np.asmatrix(df)

In [709]:
#node들 섞어줌
#np.random.seed(3)
np.random.shuffle(df)

In [710]:
#섞어서 train node 만듦
train_node1 = df[:,0:6]
train_node2 = df[:,6:12]
train_node3 = df[:,12:18]
train_node4 = df[:,18:24]
train_node5 = df[:,24:30]
ry = copy.deepcopy(df[:,30])

In [711]:
# parameters setting

y_size = 260  #numbers of y_i
lamda = 10^(-2)
eta = 10^(-4)
rho = 10^(-3)
L = 50

In [712]:
## len하면 행의 개수 나옴
y = copy.deepcopy(ry)
for i in range(y_size, len(ry)):
  y[i] = 0

In [713]:
#Hi, Li 설정하기
H1, H2, H3, H4, H5 = train_node1, train_node2, train_node3, train_node4, train_node5
I = np.identity(569)
I_L = np.identity(6)
C = np.zeros((569, 569))
for i in range(0, y_size+1):
  C[i,i]=1

In [714]:
L1, L2, L3, L4, L5 = laplacian(H1.dot(H1.transpose()), normed=True), laplacian(H2.dot(H2.transpose()), normed=True), laplacian(H3.dot(H3.transpose()), normed=True), laplacian(H4.dot(H4.transpose()), normed=True), laplacian(H5.dot(H5.transpose()), normed=True)

In [715]:
H1.shape, y.shape, I.shape, L1.shape, ry.shape, C.shape

((569, 6), (569, 1), (569, 569), (569, 569), (569, 1), (569, 569))

In [716]:
temp1 = H1.transpose().dot(H1) + eta/rho * H1.transpose().dot(L1).dot(H1) + lamda/rho * I_L
temp2 = H2.transpose().dot(H2) + eta/rho * H2.transpose().dot(L2).dot(H2) + lamda/rho * I_L
temp3 = H3.transpose().dot(H3) + eta/rho * H3.transpose().dot(L3).dot(H3) + lamda/rho * I_L
temp4 = H4.transpose().dot(H4) + eta/rho * H4.transpose().dot(L4).dot(H4) + lamda/rho * I_L
temp5 = H5.transpose().dot(H5) + eta/rho * H5.transpose().dot(L5).dot(H5) + lamda/rho * I_L

In [717]:
Q1_inv, Q2_inv, Q3_inv, Q4_inv, Q5_inv = np.linalg.inv(temp1), np.linalg.inv(temp2), np.linalg.inv(temp3), np.linalg.inv(temp4), np.linalg.inv(temp5)

In [718]:
z_bar= np.zeros((569,1))
u = np.zeros((569,1))
y_bar = np.zeros((569,1))
w1, w2, w3, w4, w5 = np.ones((6,1)), np.ones((6,1)), np.ones((6,1)), np.ones((6,1)), np.ones((6,1))

In [719]:
for i in range(0,L):
  #update wi
  w1 = Q1_inv.dot(H1.transpose()).dot(H1.dot(w1) + z_bar - y_bar - u)
  w2 = Q2_inv.dot(H2.transpose()).dot(H2.dot(w2) + z_bar - y_bar - u)
  w3 = Q3_inv.dot(H3.transpose()).dot(H3.dot(w3) + z_bar - y_bar - u)
  w4 = Q4_inv.dot(H4.transpose()).dot(H4.dot(w4) + z_bar - y_bar - u)
  w5 = Q5_inv.dot(H5.transpose()).dot(H5.dot(w5) + z_bar - y_bar - u)

  #update y_bar
  y_bar = (H1.dot(w1)+H2.dot(w2)+H3.dot(w3)+H4.dot(w4)+H5.dot(w5))/5

  #update z_bar
  z_bar = np.linalg.inv((5*C+ rho*I)).dot(C.dot(y) + rho*y_bar + rho*u)

  #update u
  u = u + y_bar - z_bar


In [720]:
y_hat = (H1.dot(w1) + H2.dot(w2) + H3.dot(w3) + H4.dot(w4) + H5.dot(w5))/5

In [721]:
y_hat = (y_hat-min(y_hat))/(max(y_hat)-min(y_hat)) + 1

In [722]:
for i in range(0, len(y_hat)):
  if y_hat[i] > 1.5:
    y_hat[i] = 2
  else:
    y_hat[i] = 1

In [723]:
error_rate = 100*(np.linalg.norm(y_hat - ry, 1))/569
error_rate

29.876977152899823